# Prompt Engineering Guide

The basic goal of prompt engineering is designing appropriate inputs for prompting methods.

- Self-Consistency

- Knowledge Generation

1. Prompt a language model using chain-of-thought (CoT) prompting
2. Replace the “greedy decode” in CoT prompting by sampling from the language model’s decoder to generate a diverse set of reasoning paths;
3. Marginalize out the reasoning paths and aggregate by choosing the most consistent answer in the final answer set.

In [9]:
from openai import OpenAI
client = OpenAI()

def get_response(prompt_question, num_of_responses=10):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}],
        n=num_of_responses,
    )
    
    return response

self_consistency_prompt = """
Q: If a library initially has 120 books and it acquires 30 more books, how many books are there in total?
A: The library starts with 120 books. It then receives an additional 30 books. Now, there are 120 + 30 = 150 books. The answer is 150.

Q: Alex has a small garden where he grows tomatoes. Each day, his plants yield 20 tomatoes. He uses 8 of them to make salads for his family’s dinner and gives 5 to his neighbor. How many tomatoes does he have left to sell or store each day?
"""

responses = get_response(self_consistency_prompt, num_of_responses=10)

In [10]:
for n in responses.choices:
    print(n.message.content)
    print("******")

To find out how many tomatoes Alex has left to sell or store each day, we can subtract the tomatoes he uses for making salads and gives to his neighbor from the total yield of his plants.

Total yield of tomatoes per day: 20 tomatoes
Tomatoes used for making salads: 8 tomatoes
Tomatoes given to the neighbor: 5 tomatoes

Tomatoes left to sell or store each day: Total yield - Tomatoes used for salads - Tomatoes given to neighbor
Tomatoes left to sell or store each day: 20 - 8 - 5
Tomatoes left to sell or store each day: 7 tomatoes

Therefore, Alex has 7 tomatoes left to sell or store each day.
******
To calculate how many tomatoes Alex has left after using some for salads and giving some to his neighbor, we need to subtract the total tomatoes used from the daily yield.

Daily yield = 20 tomatoes
Tomatoes used for salads = 8 tomatoes
Tomatoes given to neighbor = 5 tomatoes

Total tomatoes used = Tomatoes for salads + Tomatoes for neighbor
Total tomatoes used = 8 tomatoes + 5 tomatoes = 13

# Knowledge Generation

1. Knowledge generation
2. Knowledge integration

In [11]:
def generate_knowledge(question):
    # Generate background knowledge-based facts by prompting the model
    knowledge_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are an AI trained to generate facts about background knowledge."},
            {"role": "user", "content": f"Generate facts about the background knowledge for this question: {question}"}
        ]
    )
    knowledge = knowledge_response.choices[0].message.content
    
    # Use the generated knowledge to enhance the answer to the original question
    enhanced_answer_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are an AI trained to answer questions using provided knowledge."},
            {"role": "user", "content": f"Given the knowledge: {knowledge}, answer the question: {question}"}
        ]
    )
    enhanced_answer = enhanced_answer_response.choices[0].message.content
    
    return knowledge, enhanced_answer

# Example usage
question = "Why do leaves change color in autumn?"
knowledge, answer = generate_knowledge(question)
print("Generated Knowledge:", knowledge)
print("Enhanced Answer:", answer)

Generated Knowledge: 1. Leaves change color in the autumn due to the breakdown of chlorophyll, the green pigment responsible for photosynthesis. This breakdown reveals other pigments in the leaves, such as carotenoids (yellow and orange) and anthocyanins (red and purple).

2. Environmental factors such as temperature and sunlight play a role in triggering the chemical processes that cause leaves to change color. As the days become shorter and temperatures drop in the autumn, trees prepare for dormancy by shedding their leaves.

3. Different tree species exhibit a variety of colors in the autumn based on the specific pigments present in their leaves. For example, maple trees often display vibrant reds and oranges, while beech trees may have more subdued yellows and browns.

4. The timing and intensity of autumn leaf color can vary from year to year depending on factors such as weather conditions, soil moisture levels, and the overall health of the trees.

5. The colorful display of autu